In [4]:
import torch
import itertools
import numpy as np
def custom_cdist(x1, x2, types):
    """
    Compute the pairwise distance between rows of x1 and rows of x2 based on measurement types.

    Args:
        x1 (torch.Tensor): A tensor of shape (m, d)
        x2 (torch.Tensor): A tensor of shape (n, d)
        types (list): A list of measurement types for each pair of rows.
        p (float): The norm degree. Default is 2 for Euclidean distance.

    Returns:
        torch.Tensor: A tensor of shape (m, n) with the pairwise distances.
    """
    assert len(types) == x1.shape[0], "Length of types must match number of rows in x1"
    
    # Expand x1 and x2 to compute pairwise distances
    x1_expanded = x1.unsqueeze(1)  # shape: (m, 1, d)
    x2_expanded = x2.unsqueeze(0)  # shape: (1, n, d)
    
    # Compute the difference between each pair of points
    diff = x1_expanded - x2_expanded  # shape: (m, n, d)
    
    distances = torch.zeros((x1.shape[0], x2.shape[0]))
    
    for i, t in enumerate(types):
        if t in ['A', 'self']:
            # For Euclidean distance
            dist = torch.sqrt(torch.sum(diff[i] ** 2, dim=-1))
        elif t in ['B']:
            # For azimuth distance
            goal_vectors = x2 - x1[i, :2]
            goal_azimuths = torch.atan2(goal_vectors[:, 1], goal_vectors[:, 0])
            observed_azimuth = torch.atan2(x2[:, 1] - x1[i, 1], x2[:, 0] - x1[i, 0])
            relative_azimuths = torch.abs((goal_azimuths - observed_azimuth + np.pi) % (2 * np.pi) - np.pi)
            dist = 1.0 / 8 * torch.exp(-relative_azimuths / np.pi)
        else:
            raise ValueError("Unknown type")
        
        distances[i] = dist
    
    return distances

def calculate_joint_goal_probabilities_with_configs(agent_positions, goal_positions, obs_types, reward_configurations):
    """
    Calculate the joint goal probabilities for any number of agents and goals,
    applying a reward to specified configurations.

    Parameters:
    - agent_positions (tensor): Tensor of shape [num_agents, 2] representing the positions of agents.
    - goal_positions (tensor): Tensor of shape [num_goals, 2] representing the positions of goals.
    - reward_configurations (list of tuples): List of configurations to reward. Each configuration is a tuple of goal indices.
    - reward_factor (float): Factor by which to increase the probability of specified configurations.

    Returns:
    - joint_probabilities (tensor): Tensor of shape [num_goals, num_goals, ...] (num_agents times) representing the joint probabilities.
    """
    num_agents = agent_positions.shape[0]
    num_goals = goal_positions.shape[0]

    # Calculate distances between agents and goals
    distances = custom_cdist(agent_positions, goal_positions, obs_types)
    # distances = torch.cdist(agent_positions, goal_positions, p=2)  # Calculate Euclidean distances (L2 norm
    # Convert distances to probabilities using softmax
    probabilities = torch.softmax(-distances, dim=1)  # Apply softmax along the goal dimension

    # Initialize joint probabilities as a tensor of ones with the appropriate shape
    joint_probabilities = torch.ones([num_goals] * num_agents)

    # Calculate joint probabilities
    for i in range(num_agents):
        shape = [1] * num_agents
        shape[i] = num_goals
        joint_probabilities = joint_probabilities * probabilities[i].view(shape)

    # Only return the specified configurations
    likelihood = torch.zeros(len(reward_configurations), dtype=torch.float64)
    for id, config in enumerate(reward_configurations):
        likelihood[id] = joint_probabilities[config].prod()

    # Normalize the joint probabilities
    likelihood /= likelihood.sum()

    return likelihood

# Example usage
# Define the positions of agents and goals (arbitrary example positions)
agent_positions = torch.tensor([[1, 2], [4, 4], [5, 6]],dtype=float)  # 3 agents
goal_positions = torch.tensor([[2, 3], [4, 5], [6, 6]],dtype=float)  # 3 goals
obs_types = ['A', 'A', 'A']

# Define reward configurations
configurations = [(0, 0, 0), (0, 1, 0)]  # Return normalized likelihoods for (G1, G1, G1) and (G1, G2, G1)
# Generate all permutations
tuple_elements = [i for i in range(goal_positions.shape[0])]
configurations = list(itertools.permutations(tuple_elements))

# Calculate joint goal probabilities with specific rewards
joint_probabilities_with_specific_rewards = calculate_joint_goal_probabilities_with_configs(agent_positions, goal_positions, obs_types, configurations)

# Print the joint probabilities tensor with specific rewards
print(joint_probabilities_with_specific_rewards.numpy())


[8.88656219e-01 9.43555193e-02 1.52597027e-02 3.29630184e-04
 1.16243643e-03 2.36492038e-04]


In [9]:
import numpy as np

def custom_cdist(x1, x2, types, max_distance=30.0):
    """
    Compute the pairwise distance between rows of x1 and rows of x2 based on measurement types.

    Args:
        x1 (np.ndarray): An array of shape (m, d)
        x2 (np.ndarray): An array of shape (n, d)
        types (list): A list of measurement types for each pair of rows.
    Returns:
        np.ndarray: An array of shape (m, n) with the pairwise distances.
    """
    assert len(types) == x1.shape[0], "Length of types must match number of rows in x1"

    distances = np.zeros((x1.shape[0], x2.shape[0]), dtype=float)
    
    for i, t in enumerate(types):
        if t == 's': # Self-observation
            depth_multiplier = max(2.0, 2.0 / np.min(x1[i] - x2))
            dist = depth_multiplier * np.exp(-np.linalg.norm(x1[i] - x2, axis=-1) / max_distance)
        elif t == 'A':
            dist = np.exp(-np.linalg.norm(x1[i] - x2, axis=-1) / max_distance)
        elif t == 'B':
            goal_vectors = x2 - x1[i, :2]
            goal_azimuths = np.arctan2(goal_vectors[:, 1], goal_vectors[:, 0])
            observed_azimuth = np.arctan2(x2[:, 1] - x1[i, 1], x2[:, 0] - x1[i, 0])
            relative_azimuths = np.abs((goal_azimuths - observed_azimuth + np.pi) % (2 * np.pi) - np.pi)
            dist = 1.0 / 8 * np.exp(-relative_azimuths / np.pi)
        else:
            dist = np.zeros(x2.shape[0], dtype=x1.dtype)
        
        distances[i] = dist
    
    return distances

def calculate_joint_goal_probs(agent_poses, goals, predict_types, reward_configs, max_distance=30.0):
    """
    Calculate the joint goal probabilities for any number of agents and goals,
    applying a reward to specified configurations.

    Parameters:
    - agent_poses (np.ndarray): Array of shape [num_agents, 2] representing the positions of agents.
    - goals (np.ndarray): Array of shape [num_goals, 2] representing the positions of goals.
    - predict_types (list): List of types for prediction
    - reward_configs (list of tuples): List of configurations to reward. Each configuration is a tuple of goal indices.
    
    Returns:
    - joint_probabilities (np.ndarray): Array representing the joint probabilities.
    """
    num_agents = agent_poses.shape[0]
    num_goals = goals.shape[0]

    # Calculate distances between agents and goals
    distances = custom_cdist(agent_poses, goals, predict_types, max_distance)

    # Convert distances to probabilities using softmax
    probabilities = np.exp(distances) / np.exp(distances).sum(axis=1, keepdims=True)  # Apply softmax along the goal dimension

    # Initialize joint probabilities as an array of ones with the appropriate shape
    joint_probabilities = np.ones([num_goals] * num_agents, dtype=probabilities.dtype)

    # Calculate joint probabilities
    for i in range(num_agents):
        joint_probabilities *= probabilities[i].reshape([num_goals if j == i else 1 for j in range(num_agents)]) 

    # Only return the specified configurations
    likelihood = np.array([joint_probabilities[tuple(config)] for config in reward_configs], dtype=np.float64)

    # Normalize the joint probabilities
    likelihood /= likelihood.sum()

    return likelihood

# Example usage:
agent_poses = np.array([[1, 2], [3, 4]])
goals = np.array([[5, 6], [7, 8], [9, 10]])
predict_types = ['A', 'B']
reward_configs = [(0, 1), (1, 2), (1,1)]

likelihood = calculate_joint_goal_probs(agent_poses, goals, predict_types, reward_configs)
print(likelihood)

[0.35009169 0.32495415 0.32495415]
